In [52]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import  Dense
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
import math

In [53]:
df1 = pd.read_csv("powerconsumption.csv", index_col='Datetime', parse_dates=['Datetime'])
df2 = pd.read_csv("/content/Weather_Data.csv", index_col='Date', parse_dates=['Date'])
df1

FileNotFoundError: [Errno 2] No such file or directory: 'powerconsumption.csv'

In [ ]:
# prompt: make df1 do per hour

df1_H = df1.resample('D').mean()
df1_H

In [ ]:
data1 = df1_H.iloc[:,0:2]

In [ ]:
data1

In [ ]:
# prompt: extract only 2017 data from df2

df2_2017 = df2[df2.index.year == 2017]


In [ ]:
df2_2017

In [ ]:
data2 = df2_2017.iloc[:,4:8]
data2

In [ ]:
# prompt: concat data1 and data2

df_concat = pd.concat([data1, data2], axis=1)
df_concat


In [ ]:
df_concat = pd.concat([df_concat, df1_H.iloc[:,-1]], axis=1)
df_concat

In [ ]:
# prompt: remove null

df_concat = df_concat.dropna()
df_concat

In [ ]:
# prompt: change last column name to 'weight'

df_concat = df_concat.rename(columns={df_concat.columns[-1]: 'weight'})


In [ ]:
df_concat

#Start here

In [94]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import  Dense
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
import math

In [95]:
df_concat = pd.read_csv("/content/waterData.csv")
df_concat = df_concat.drop(columns=['Unnamed: 0'])
df_concat

,Temperature,Humidity,Rainfall,RainToday,Evaporation,Sunshine,weight
0,9.675299,68.519306,0.0,No,4.0,11.1,17868.795181
1,12.476875,71.456319,0.0,No,3.8,11.0,17820.763053
2,12.100000,74.981667,0.0,No,2.6,6.8,17620.803213
3,10.509479,75.459792,49.4,Yes,2.4,5.7,17673.694779
4,10.866444,71.040486,0.2,No,0.4,6.3,17664.176707
...,...,...,...,...,...,...,...
359,11.621840,69.070903,0.0,No,3.6,9.1,11395.678272
360,15.232917,59.445903,0.0,No,4.0,8.5,11498.279312
361,13.662361,62.839375,0.0,No,4.0,9.1,11167.026811
362,12.990486,49.078750,0.0,Yes,2.2,1.3,11171.268507


In [96]:
# prompt: make the column 'Rainfall' = 'Rainfall' + 'Humidity' * '3

df_concat['Rainfall'] = df_concat['Rainfall'] + df_concat['Humidity'] * 3


In [97]:
df_concat['RainToday'] = df_concat['Rainfall']/2 + df_concat['Humidity']

In [98]:
df_concat

,Temperature,Humidity,Rainfall,RainToday,Evaporation,Sunshine,weight
0,9.675299,68.519306,205.557917,171.298264,4.0,11.1,17868.795181
1,12.476875,71.456319,214.368958,178.640799,3.8,11.0,17820.763053
2,12.100000,74.981667,224.945000,187.454167,2.6,6.8,17620.803213
3,10.509479,75.459792,275.779375,213.349479,2.4,5.7,17673.694779
4,10.866444,71.040486,213.321458,177.701215,0.4,6.3,17664.176707
...,...,...,...,...,...,...,...
359,11.621840,69.070903,207.212708,172.677257,3.6,9.1,11395.678272
360,15.232917,59.445903,178.337708,148.614757,4.0,8.5,11498.279312
361,13.662361,62.839375,188.518125,157.098437,4.0,9.1,11167.026811
362,12.990486,49.078750,147.236250,122.696875,2.2,1.3,11171.268507


In [99]:
# prompt: change the column names to
# nfrastructure Costs Ic
# Energy Consumption Ec
# Carbon foot print Cfp
# Water Leakage "Average amount per day for exp" L_amount
# water Leakage likelyhood "a forcast " Ll
# Distance or Length D
# Condition of Routes  Cr

df_concat = df_concat.rename(columns={
    'weight': 'Ic',
    'Rainfall': 'Ec',
    'RainToday': 'Cfp',
    'Humidity': 'L_amount',
    'Evaporation': 'Ll',
    'Temperature': 'D',
    'Sunshine': 'Cr'
})

df_concat


,D,L_amount,Ec,Cfp,Ll,Cr,Ic
0,9.675299,68.519306,205.557917,171.298264,4.0,11.1,17868.795181
1,12.476875,71.456319,214.368958,178.640799,3.8,11.0,17820.763053
2,12.100000,74.981667,224.945000,187.454167,2.6,6.8,17620.803213
3,10.509479,75.459792,275.779375,213.349479,2.4,5.7,17673.694779
4,10.866444,71.040486,213.321458,177.701215,0.4,6.3,17664.176707
...,...,...,...,...,...,...,...
359,11.621840,69.070903,207.212708,172.677257,3.6,9.1,11395.678272
360,15.232917,59.445903,178.337708,148.614757,4.0,8.5,11498.279312
361,13.662361,62.839375,188.518125,157.098437,4.0,9.1,11167.026811
362,12.990486,49.078750,147.236250,122.696875,2.2,1.3,11171.268507


In [100]:
# prompt: download csv
'''
from google.colab import files

df_concat.to_csv('waterData.csv')
files.download('waterData.csv')'''


"\nfrom google.colab import files\n\ndf_concat.to_csv('waterData.csv')\nfiles.download('waterData.csv')"

In [101]:
'''import numpy as np

# Function to input pairwise comparisons
def input_comparison(criteria):
    n = len(criteria)
    comparison_matrix = np.ones((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            print(f"Comparing {criteria[i]} and {criteria[j]}:")
            comparison = float(input(f"Enter the importance of {criteria[i]} compared to {criteria[j]} (1-9): "))
            comparison_matrix[i, j] = comparison
            comparison_matrix[j, i] = 1 / comparison
    return comparison_matrix

# Criteria
criteria = ['Ic', 'Ec', 'Cfp', 'L_amount', 'Ll', 'D', 'Cr']

# Input pairwise comparisons
comparison_matrix = input_comparison(criteria)

# Calculate weights
eigenvalues, eigenvectors = np.linalg.eig(comparison_matrix)
weights = eigenvectors[:, np.argmax(eigenvalues)]

# Normalize weights
weights /= np.sum(weights)

# Print the weights
for i in range(len(criteria)):
    print(f"Weight for {criteria[i]}: {weights[i]}")'''

'import numpy as np\n\n# Function to input pairwise comparisons\ndef input_comparison(criteria):\n    n = len(criteria)\n    comparison_matrix = np.ones((n, n))\n    for i in range(n):\n        for j in range(i + 1, n):\n            print(f"Comparing {criteria[i]} and {criteria[j]}:")\n            comparison = float(input(f"Enter the importance of {criteria[i]} compared to {criteria[j]} (1-9): "))\n            comparison_matrix[i, j] = comparison\n            comparison_matrix[j, i] = 1 / comparison\n    return comparison_matrix\n\n# Criteria\ncriteria = [\'Ic\', \'Ec\', \'Cfp\', \'L_amount\', \'Ll\', \'D\', \'Cr\']\n\n# Input pairwise comparisons\ncomparison_matrix = input_comparison(criteria)\n\n# Calculate weights\neigenvalues, eigenvectors = np.linalg.eig(comparison_matrix)\nweights = eigenvectors[:, np.argmax(eigenvalues)]\n\n# Normalize weights\nweights /= np.sum(weights)\n\n# Print the weights\nfor i in range(len(criteria)):\n    print(f"Weight for {criteria[i]}: {weights[i]}")'

In [102]:
import numpy as np

# Function to calculate the Saaty scale values
def saaty_scale(n):
    scale = np.ones((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            scale[i, j] = 1 / (j - i)
            scale[j, i] = 1 / scale[i, j]
    return scale

# Criteria
criteria = ['Ic', 'Ec', 'Cfp', 'L_amount', 'Ll', 'D', 'Cr']
n = len(criteria)

# Calculate Saaty scale values
scale = saaty_scale(n)

# Derive pairwise comparison matrix
comparison_matrix = scale

# Calculate weights
eigenvalues, eigenvectors = np.linalg.eig(comparison_matrix)
weights = eigenvectors[:, np.argmax(eigenvalues)]

# Normalize weights
weights /= np.sum(weights)

# Print the weights
for i in range(n):
    print(f"Weight for {criteria[i]}: {weights[i]}")

Weight for Ic: (0.04538040325157452+0j)
Weight for Ec: (0.058923741042928274+0j)
Weight for Cfp: (0.08232971373007834+0j)
Weight for L_amount: (0.1177017150934558+0j)
Weight for Ll: (0.16722614873528738+0j)
Weight for D: (0.23101904346181995+0j)
Weight for Cr: (0.29741923468485576+0j)


In [103]:


df_concat['TotalCostAuto'] = df_concat['Ic'] * weights[0] + df_concat['Ec'] * weights[1] + df_concat['Cfp'] * weights[2] + df_concat['L_amount'] * weights[3] + df_concat['Ll'] * weights[4] + df_concat['D'] * weights[5] + df_concat['Cr'] * weights[6]
df_concat['TotalCostAuto'] = df_concat['TotalCostAuto'].abs()


In [104]:
df_concat

,D,L_amount,Ec,Cfp,Ll,Cr,Ic,TotalCostAuto
0,9.675299,68.519306,205.557917,171.298264,4.0,11.1,17868.795181,851.378586
1,12.476875,71.456319,214.368958,178.640799,3.8,11.0,17820.763053,851.252278
2,12.100000,74.981667,224.945000,187.454167,2.6,6.8,17620.803213,842.404844
3,10.509479,75.459792,275.779375,213.349479,2.4,5.7,17673.694779,849.260619
4,10.866444,71.040486,213.321458,177.701215,0.4,6.3,17664.176707,841.619825
...,...,...,...,...,...,...,...,...
359,11.621840,69.070903,207.212708,172.677257,3.6,9.1,11395.678272,557.689852
360,15.232917,59.445903,178.337708,148.614757,4.0,8.5,11498.279312,558.253234
361,13.662361,62.839375,188.518125,157.098437,4.0,9.1,11167.026811,544.734230
362,12.990486,49.078750,147.236250,122.696875,2.2,1.3,11171.268507,535.266224


In [111]:
weights1 = []
for i in criteria:
  k = float(input(f"Enter the percentage for {i} (0-1) "))
  weights1.append(k)
if sum(weights1) != 1:
    print("the percentage is not correct",sum(weights1))
    for i in range(len(weights1)):
      weights1[i] = 0
df_concat['TotalCostM'] = df_concat['Ic'] * weights1[0] + df_concat['Ec'] * weights1[1] + df_concat['Cfp'] * weights1[2] + df_concat['L_amount'] * weights1[3] + df_concat['Ll'] * weights1[4] + df_concat['D'] * weights1[5] + df_concat['Cr'] * weights1[6]

Enter the percentage for Ic (0-1) 0.1
Enter the percentage for Ec (0-1) 0.1
Enter the percentage for Cfp (0-1) 0.1
Enter the percentage for L_amount (0-1) 0.1
Enter the percentage for Ll (0-1) 0.1
Enter the percentage for D (0-1) 0.1
Enter the percentage for Cr (0-1) 0.4


In [113]:
df_concat

,D,L_amount,Ec,Cfp,Ll,Cr,Ic,TotalCostAuto,TotalCostM
0,9.675299,68.519306,205.557917,171.298264,4.0,11.1,17868.795181,851.378586,1837.224597
1,12.476875,71.456319,214.368958,178.640799,3.8,11.0,17820.763053,851.252278,1834.550600
2,12.100000,74.981667,224.945000,187.454167,2.6,6.8,17620.803213,842.404844,1815.008405
3,10.509479,75.459792,275.779375,213.349479,2.4,5.7,17673.694779,849.260619,1827.399290
4,10.866444,71.040486,213.321458,177.701215,0.4,6.3,17664.176707,841.619825,1816.270631
...,...,...,...,...,...,...,...,...,...
359,11.621840,69.070903,207.212708,172.677257,3.6,9.1,11395.678272,557.689852,1189.626098
360,15.232917,59.445903,178.337708,148.614757,4.0,8.5,11498.279312,558.253234,1193.791060
361,13.662361,62.839375,188.518125,157.098437,4.0,9.1,11167.026811,544.734230,1162.954511
362,12.990486,49.078750,147.236250,122.696875,2.2,1.3,11171.268507,535.266224,1151.067087


In [114]:
from google.colab import files

df_concat.to_csv('waterData1.csv')
files.download('waterData1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>